# Install Requirements

In [ ]:
# !pip -qq install  peft
# !pip -qq install  bitsandbytes
# !pip -qq install  trl

In [ ]:
# !pip -qq install git+https://github.com/huggingface/transformers

# Import Libraries

In [1]:
import json
import re
from pprint import pprint
import time

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login

from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM , AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer , BitsAndBytesConfig

from trl import SFTTrainer

DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

# Login HuggingFace

In [ ]:
notebook_login()

# load dataset

Salesforce/dialogstudio: "TweetSumm"

In [2]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [3]:
print(dataset['train'][0])

{'original dialog id': 'b065262210783596c1fe79466b8f8985', 'new dialog id': 'TweetSumm--train--1', 'dialog index': 1, 'original dialog info': '{"summaries": {"extractive_summaries": [[{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}, {"is_agent": false, "sentences": ["@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1."]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Ap

# Preprocess the dataset

In [4]:

# Default prompt that instructs the model to generate a summary.
DEFAULT_SYSTEM_PROMPT = """
Please provide a concise and accurate summary of the following conversation between a human and an AI agent.
""".strip()

class DatasetProcessor:
    def __init__(self, data):
        # Initialize the processor with the dataset.
        self.data = data

    @staticmethod
    def clean_text(text):
        """
        Clean the input text by removing URLs, user mentions, and extra whitespaces.
        
        Args:
            text (str): The text to be cleaned.

        Returns:
            str: The cleaned text.
        """
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"@[^\s]+", "", text)
        text = re.sub(r"\s+", " ", text)
        return re.sub(r"\^[^ ]+", "", text)

    @staticmethod
    def create_conversation_text(data_point):
        """
        Construct the conversation text from a single data point's dialog log.
        
        Args:
            data_point (dict): A single entry from the dataset.

        Returns:
            str: The formatted conversation text.
        """
        text = ""
        for item in data_point["log"]:
            user = DatasetProcessor.clean_text(item["user utterance"])
            text += f"user: {user.strip()}\n"

            agent = DatasetProcessor.clean_text(item["system response"])
            text += f"agent: {agent.strip()}\n"

        return text

    @staticmethod
    def generate_training_prompt(conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT):
        """
        Generate a formatted training prompt that includes the system prompt, the input conversation, and the expected response.
        
        Args:
            conversation (str): The conversation text.
            summary (str): The summary of the conversation.
            system_prompt (str): Instructional text guiding the summary generation.

        Returns:
            str: The formatted training prompt.
        """
        return f"""### Instruction: {system_prompt}

        ### Input:
        {conversation.strip()}

        """.strip()

    @staticmethod
    def generate_text(data_point):
        """
        Generate the text components required for training, including the conversation text and its summary.
        
        Args:
            data_point (dict): A single entry from the dataset.

        Returns:
            dict: A dictionary containing the conversation, the summary, and the full training text.
        """
        summaries = json.loads(data_point["original dialog info"])["summaries"]["abstractive_summaries"]
        summary = " ".join(summaries[0])

        conversation_text = DatasetProcessor.create_conversation_text(data_point)
        return {
            "conversation": conversation_text,
            "summary": summary,
            "new_prompt": DatasetProcessor.generate_training_prompt(conversation_text, summary),
        }

    def process(self):
        """
        Process the entire dataset by shuffling, generating text for each entry, and removing unnecessary columns.
        
        Returns:
            Dataset: The processed dataset ready for use in training or evaluation.
        """
        process_func = lambda x: DatasetProcessor.generate_text(x)
        self.data = (
            self.data.shuffle(seed=42)
            .map(process_func)
            .remove_columns(
                [
                    "original dialog id",
                    "new dialog id",
                    "dialog index",
                    "original dialog info",
                    "log",
                    "prompt",
                ]
            )
        )
        return self.data


In [5]:
# Processing the dataset
processor = DatasetProcessor(dataset["train"])
dataset["train"] = processor.process()

processor = DatasetProcessor(dataset["validation"])
dataset["validation"] = processor.process()

processor = DatasetProcessor(dataset["test"])
dataset["test"] = processor.process()

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation', 'summary', 'new_prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['conversation', 'summary', 'new_prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['conversation', 'summary', 'new_prompt'],
        num_rows: 110
    })
})

In [7]:
dataset["train"][0]

{'conversation': 'user: Do you have a plan to notify passengers well in advance of pilot related cancellations or just wait til the day before? Will you protect passengers on other airlines if flights are cancelled b/c of pilot shortages?\nagent: We\'re planning to fly as scheduled, Shaun.\nuser: HOW ABOUT ANSWERING MY QUESTION. I\'m asking if you do not get enough pilots to fly, which is a possibility, do you have a contingency plan in place on how to get customers to their destinations &amp; when will it be relayed to customers. THE DAY BEFORE WILL NOT BE ACCEPTABLE!\nagent: Our team is working hard to avoid cancellations and you\'ll be notified if otherwise.\nuser: Your reading comprehension is terrible. WHEN WILL WE BE NOTIFIED? 3 hours b4 our flight so all other flights r sold out? Instead of Doug Parker making comments like "I don\'t think we\'re ever going to lose money again," he should b assuring customers we\'re getting home 4 XMAS\nagent: As of now, flights are scheduled and

In [8]:
# Load tokenizer and model 

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-3b-4e1t", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-3b-4e1t", torch_dtype=torch.bfloat16,use_auth_token=True, trust_remote_code=True)

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:703: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [9]:
# Create dataframes from the dataset
train_df = pd.DataFrame(dataset["train"])
validation_df = pd.DataFrame(dataset["validation"])
test_df = pd.DataFrame(dataset["test"])

# Save dataframes to CSV files
train_df.to_csv("train.csv", index=False, columns=["conversation", "new_prompt", "summary"])
validation_df.to_csv("validation.csv", index=False, columns=["conversation", "new_prompt", "summary"])
test_df.to_csv("test.csv", index=False, columns=["conversation", "new_prompt", "summary"])

In [10]:
model = model.to("cuda:1")

# In Context Learning

In [11]:
model.eval()
print("model loaded")

model loaded


In [12]:
dataset["test"][30]

{'conversation': "user: On went to buy a Lip na mpesa wrist band in your shop in Thika and was sent to Annas Mall where i dint find your Team!\nagent: Apologies for this, check with any M-PESA Agent, or Safaricom authorized dealer. You may also go back to our shop in Thika -(contd)to verify whether we have restocked.\nuser: I was at your shop at Thika and they told me they were not sure where the guys dealing with the wrist bands were\nagent: The mpesa 1 tap is not available at Thika its being piloted in Nairobi, Mombasa, Kisumu, Eldoret and Nyeri towns ... cont... currently. To get it you may visit the nearest town which is Nairobi to purchase one at a cost of 20/-.\nuser: Would you please confirm for me if I will get the wrist band at your shop in Moi Avenue today so I don't have to travel for nothing.\nagent: Hi, yes you will and at the moment it can only be used in the towns mentioned in our previous response only.\nuser: On Tuesday I went to your shop on Moi Avenue to get wrist ba

# Zero Shot

In [13]:
index = 100
dialogue = dataset['test'][index]['conversation']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Please provide a concise and accurate summary of the following conversation between a human and an AI agent.
{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 

generation_config = GenerationConfig(
    max_length=2048, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id  ,
    temperature=0.7,

)

original_model_outputs = model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')



/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Customer is asking help that  how to remove red eye in ligth room cc even he cant find it in tool  and even customer want some new advance features. Agent is giving details on it and then sends a link where he can get help and also asked customer to  report a  complaint where  his engineer team will  get alert and help him over it.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:

Please provide a concise and accurate summary of the following conversation between a human and an AI agent.
user: Can you tell me how to do Red Eye Removal in Lightroom CC? I just moved to it and don't see the Red Eye Removal tool.
agent: Hi Bob, here is a link to show you to use the Red eye removal in Lightroom CC.
user: Does not apply to the NEW LightRoom CC. Any other suggestions?
agent: Bob, I will loop in our Lightr

## One-Shot

In [14]:

# Select a random conversation-summary pair from the train dataset for the one-shot example
example_conversation_train = dataset['train'][0]['conversation']
example_summary_train =  dataset['train'][0]['summary']

# Select the conversation and the human-generated summary from the test dataset
index = 100  
dialogue_test = dataset['test'][index]['conversation']
baseline_human_summary = dataset['test'][index]['summary']


# Prepare the one-shot prompt
prompt = f"""
### Example Conversation:
{example_conversation_train}

### Example Summary:
{example_summary_train}

---

### New Conversation:
{dialogue_test}

### Please provide a concise and accurate summary for the new conversation above:
"""

input_ids = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 

generation_config = GenerationConfig(
    max_length=1024, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
)

# Generate the summary for the new conversation
original_model_outputs = model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

# Output the results
dash_line = '-' * 100
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ONE-SHOT MODEL OUTPUT:\n{original_model_text_output}')


----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Customer is asking help that  how to remove red eye in ligth room cc even he cant find it in tool  and even customer want some new advance features. Agent is giving details on it and then sends a link where he can get help and also asked customer to  report a  complaint where  his engineer team will  get alert and help him over it.
----------------------------------------------------------------------------------------------------
ONE-SHOT MODEL OUTPUT:

### Example Conversation:
user: Do you have a plan to notify passengers well in advance of pilot related cancellations or just wait til the day before? Will you protect passengers on other airlines if flights are cancelled b/c of pilot shortages?
agent: We're planning to fly as scheduled, Shaun.
user: HOW ABOUT ANSWERING MY QUESTION. I'm asking if you do not get enough pilots to fly, which is a possibility, do yo

# Fine Tuning Model

In [2]:
## load again model and tokenizer

tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-3b-4e1t", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-3b-4e1t", torch_dtype=torch.bfloat16,use_auth_token=True, trust_remote_code=True)


# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.float16,
#     )

# model = AutoModelForCausalLM.from_pretrained(
#     "stabilityai/stablelm-3b-4e1t",
#     use_safetensors=True,
#     quantization_config=bnb_config,
#     trust_remote_code=True,
#     use_auth_token=True,
#     torch_dtype=torch.bfloat16,
# )

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:703: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [2]:
# load dataset from csvs

dataset = load_dataset('csv', data_files={
    'train': 'train.csv',
    'validation': 'validation.csv',
    'test': 'test.csv'
})


In [4]:
# Add [PAD] as the padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [5]:

def tokenize_prompts_and_summaries(examples):
    """
    Tokenizes the prompts and summaries from the given examples.

    This function takes a dictionary `examples` with keys 'new_prompt' and 'summary'. It tokenizes both the prompts and summaries,
    pads them to a maximum length, and truncates if necessary. The tokenized inputs are then added to the `examples` dictionary
    under the keys 'input_ids' for the prompts and 'labels' for the summaries.

    Parameters:
    examples (dict): A dictionary containing 'new_prompt' and 'summary' keys. 'new_prompt' is a list of prompts, and 'summary' is a list of corresponding summaries.

    Returns:
    dict: The original `examples` dictionary updated with 'input_ids' for tokenized prompts and 'labels' for tokenized summaries.
    """

    # Define the end of the prompt token
    end_prompt_token = '\n\nSummary: '

    # Concatenate end of prompt token with each new prompt
    prompts = [new_prompt + end_prompt_token for new_prompt in examples["new_prompt"]]

    # Tokenizing the prompts and the summaries
    tokenized_inputs = tokenizer(prompts, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
    tokenized_labels = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=1024, return_tensors="pt")

    # Extracting input_ids for inputs and labels
    examples['input_ids'] = tokenized_inputs.input_ids
    examples['labels'] = tokenized_labels.input_ids

    return examples


# Apply the function to the dataset
tokenized_datasets = dataset.map(tokenize_prompts_and_summaries, batched=True)

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

In [6]:
# remove columns
tokenized_datasets = tokenized_datasets.remove_columns(['conversation', 'summary' , 'new_prompt'])

In [7]:
######################### if dont want to load from csv file 

In [8]:
# def tokenize_function(example):
#     example['input_ids'] = tokenizer(example["new_prompt"], padding="max_length", truncation=True, return_tensors="pt").input_ids
#     example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

#     return example

# tokenized_datasets = dataset.map(tokenize_function)
# tokenized_datasets = tokenized_datasets.remove_columns(['conversation', 'new_prompt', 'summary'])


In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (879, 2)
Validation: (110, 2)
Test: (110, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 110
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 110
    })
})


In [10]:
def count_model_parameters(model):
    """
    Counts the total and trainable parameters of the given model.

    This function iterates through all parameters of the provided model, counting the total number of parameters
    and the number of trainable parameters (parameters with requires_grad=True).

    Parameters:
    model (torch.nn.Module): The model whose parameters are to be counted.

    Returns:
    tuple: A tuple containing the number of trainable parameters and the total number of parameters.
    """
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    return trainable_params, total_params

def print_model_parameters_info(model):
    """
    Prints the number of total and trainable parameters of the given model, along with the percentage of trainable parameters.

    Parameters:
    model (torch.nn.Module): The model whose parameter information is to be printed.
    """
    trainable_params, total_params = count_model_parameters(model)
    percentage_trainable = 100 * trainable_params / total_params
    info_message = (
        f"Trainable model parameters: {trainable_params}\n"
        f"All model parameters: {total_params}\n"
        f"Percentage of trainable model parameters: {percentage_trainable:.2f}%"
    )
    print(info_message)


In [11]:
# print model info 
print_model_parameters_info(model)

Trainable model parameters: 2795443200
All model parameters: 2795443200
Percentage of trainable model parameters: 100.00%


In [12]:
from peft import LoraConfig, get_peft_model, TaskType

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "k_proj",
    "v_proj"
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)


In [13]:
peft_model = get_peft_model(model,
                            peft_config)

# Print peft info
print_model_parameters_info(peft_model)

Trainable model parameters: 7864320
All model parameters: 2803307520
Percentage of trainable model parameters: 0.28%


In [15]:
peft_training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    save_steps=500,
    eval_steps=500,
    weight_decay=0.01,
    report_to=["tensorboard"],
    logging_dir='./logs',
)


peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"]
)


In [ ]:
peft_trainer.train()

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,8.169900


/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
peft_model_path="./peft-model-checkpoint-new"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

# Test Fine-Tuned Peft Model

In [3]:
# load model again
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stablelm-3b-4e1t", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("stabilityai/stablelm-3b-4e1t", torch_dtype=torch.bfloat16,use_auth_token=True, trust_remote_code=True)

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:703: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [4]:
# Base Model to Cuda
model.to("cuda:1")
model.eval()
print("model loaded")

model loaded


In [5]:
from peft import LoraConfig, get_peft_model, TaskType

lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "v_proj"
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

peft_model = get_peft_model(model,
                            peft_config)


In [7]:
# Load PEFT Fine Tuned Model

peft_model = PeftModel.from_pretrained(peft_model, 
                                       './peft-model-checkpoint-new/', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [8]:
peft_model.to("cuda:1")
peft_model.eval()
print("peft_model loaded")

peft_model loaded


In [10]:
index = 100
dialogue = dataset['test'][index]['conversation']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 

generation_config = GenerationConfig(
    max_length=512, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id  ,
    temperature=0,

)

peft_model_outputs = peft_model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Customer is asking help that  how to remove red eye in ligth room cc even he cant find it in tool  and even customer want some new advance features. Agent is giving details on it and then sends a link where he can get help and also asked customer to  report a  complaint where  his engineer team will  get alert and help him over it.
---------------------------------------------------------------------------------------------------
PEFT MODEL: 
Summarize the following conversation.

user: Can you tell me how to do Red Eye Removal in Lightroom CC? I just moved to it and don't see the Red Eye Removal tool.
agent: Hi Bob, here is a link to show you to use the Red eye removal in Lightroom CC.
user: Does not apply to the NEW LightRoom CC. Any other suggestions?
agent: Bob, I will loop in our Lightroom expert to help you with this. The setting may have moved to a differen

In [12]:
dialogues = dataset['test'][0:10]['conversation']
human_baseline_summaries = dataset['test'][0:10]['summary']

peft_model_summaries = []
generation_config = GenerationConfig(
    max_length=512, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id  ,
    temperature=0,

)
for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt")
    inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 


    human_baseline_text_output = human_baseline_summaries[idx]
    

    peft_model_outputs = peft_model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    last_summary_part = peft_model_text_output.split('Summary: ')[-1]  # Get the last part of the summary

    peft_model_summaries.append(last_summary_part)

zipped_summaries = list(zip(human_baseline_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries','peft_model_summaries'])
df

,human_baseline_summaries,peft_model_summaries
0,Customer is looking to change the flight on Fr...,\nThe agent is asking the user to call to make...
1,Customer is unable to use an app. Agent inform...,\nThe user is having trouble logging in to the...
2,Customer is complaining that he is unable to p...,\nThe user was having an issue with being kick...
3,Customer is saying that headphones are not wor...,\nThe conversation was about a user's problem ...
4,Customer booked the flight but with wrong birt...,\nI have a conversation with the agent and I h...
5,Customer is complaining that the watchlist is ...,\nThe conversation is about a user's watchlist...
6,Customer is complaining about customer service...,\nThe user is complaining about Sprint's custo...
7,Customer is enquiring about streaming music wi...,\nThe agent was able to summarize the conversa...
8,Customer is reporting to watch live programs w...,\n
9,The customer is complaining that he facing som...,\nThe conversation is about a store that has h...


In [13]:
import evaluate

In [14]:
rouge = evaluate.load('rouge')


In [15]:
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT MODEL:')
print(peft_model_results)

PEFT MODEL:
{'rouge1': 0.12776677911580778, 'rouge2': 0.03871861188881383, 'rougeL': 0.10721665788214924, 'rougeLsum': 0.10683639087848884}


# PEFT Model Zero-Shot Result 

In [16]:
index = 100
dialogue = dataset['test'][index]['conversation']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Please provide a concise and accurate summary of the following conversation between a human and an AI agent.
{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 

generation_config = GenerationConfig(
    max_length=2048, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id  ,
    temperature=0.7,

)

peft_model_outputs = peft_model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'Peft Model Zero shot MODEL:\n{peft_model_text_output}')

/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/root/miniconda3/envs/whisper_asr/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Customer is asking help that  how to remove red eye in ligth room cc even he cant find it in tool  and even customer want some new advance features. Agent is giving details on it and then sends a link where he can get help and also asked customer to  report a  complaint where  his engineer team will  get alert and help him over it.
---------------------------------------------------------------------------------------------------
Peft Model Zero shot MODEL:

Please provide a concise and accurate summary of the following conversation between a human and an AI agent.
user: Can you tell me how to do Red Eye Removal in Lightroom CC? I just moved to it and don't see the Red Eye Removal tool.
agent: Hi Bob, here is a link to show you to use the Red eye removal in Lightroom CC.
user: Does not apply to the NEW LightRoom CC. Any other suggestions?
agent: Bob, I will loop i

# PEFT Model One-Shot Result 

In [17]:

# Select a random conversation-summary pair from the train dataset for the one-shot example
example_conversation_train = dataset['train'][0]['conversation']
example_summary_train =  dataset['train'][0]['summary']

# Select the conversation and the human-generated summary from the test dataset
index = 100  
dialogue_test = dataset['test'][index]['conversation']
baseline_human_summary = dataset['test'][index]['summary']


# Prepare the one-shot prompt
prompt = f"""
### Example Conversation:
{example_conversation_train}

### Example Summary:
{example_summary_train}

---

### New Conversation:
{dialogue_test}

### Please provide a concise and accurate summary for the new conversation above:
"""

input_ids = tokenizer(prompt, return_tensors="pt")
inputs = {key: value.to("cuda:1") for key, value in input_ids.items()} 

generation_config = GenerationConfig(
    max_length=1024, 
    num_beams=1,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
)

# Generate the summary for the new conversation
peft_model_outputs = peft_model.generate(input_ids=inputs["input_ids"], generation_config=generation_config)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

# Output the results
dash_line = '-' * 100
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{baseline_human_summary}')
print(dash_line)
print(f'ONE-SHOT MODEL OUTPUT:\n{peft_model_text_output}')


----------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Customer is asking help that  how to remove red eye in ligth room cc even he cant find it in tool  and even customer want some new advance features. Agent is giving details on it and then sends a link where he can get help and also asked customer to  report a  complaint where  his engineer team will  get alert and help him over it.
----------------------------------------------------------------------------------------------------
ONE-SHOT MODEL OUTPUT:

### Example Conversation:
user: Do you have a plan to notify passengers well in advance of pilot related cancellations or just wait til the day before? Will you protect passengers on other airlines if flights are cancelled b/c of pilot shortages?
agent: We're planning to fly as scheduled, Shaun.
user: HOW ABOUT ANSWERING MY QUESTION. I'm asking if you do not get enough pilots to fly, which is a possibility, do yo